In [18]:
import datasets
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [7]:
ds_202001 = datasets.load_dataset('RealTimeData/bbc_news_alltime', '2020-01')
ds_202401 = datasets.load_dataset('RealTimeData/bbc_news_alltime', '2024-01')

Generating train split:   0%|          | 0/1230 [00:00<?, ? examples/s]

In [15]:
ds_202001["train"].to_csv('bbc_news_2020-01.csv', index=False)
ds_202401["train"].to_csv('bbc_news_2024-01.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

4689193

In [20]:
import re

X = list(pd.read_csv('bbc_news_2024-01.csv').itertuples())
unseen_X_train, X_val_test, _, _ = train_test_split(X, len(X)*[1], test_size=0.2, random_state=42)
unseen_X_val, unseen_X_test, _, _ = train_test_split(X_val_test, len(X_val_test)*[1], test_size=0.5, random_state=42)

X = list(pd.read_csv('bbc_news_2020-01.csv').itertuples())
seen_X_train, X_val_test, _, _ = train_test_split(X, len(X)*[1], test_size=0.2, random_state=42)
seen_X_val, seen_X_test, _, _ = train_test_split(X_val_test, len(X_val_test)*[1], test_size=0.5, random_state=42)

idx = 0
with open('train_comprehensive.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=["id", "question", "label"])
    writer.writeheader()
    
    for raw in unseen_X_train:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"train_{idx}", "question": f"Tell me more about the news: {title}", "label": "unseen"})
        idx += 1

    for raw in seen_X_train:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"train_{idx}", "question": f"Tell me more about the news: {title}", "label": "seen"})
        idx += 1
    
    
with open('val_comprehensive.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=["id", "question", "label"])
    writer.writeheader()
    
    for raw in unseen_X_val:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"val_{idx}", "question": f"Tell me more about the news: {title}", "label": "unseen"})
        idx += 1

    for raw in seen_X_val:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"val_{idx}", "question": f"Tell me more about the news: {title}", "label": "seen"})
        idx += 1

with open('test_comprehensive.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=["id", "question", "label"])
    writer.writeheader()
    
    for raw in unseen_X_test:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"test_{idx}", "question": f"Tell me more about the news: {title}", "label": "unseen"})
        idx += 1

    for raw in seen_X_test:
        title = re.sub("-* *BBC News", "", raw.title).strip()
        writer.writerow({"id": f"test_{idx}", "question": f"Tell me more about the news: {title}", "label": "seen"})
        idx += 1


In [ ]:

conda deactivate
conda activate value
cd ~/Hallu_source/generate

GPUID=1
MODEL='Llama2-7B-Chat'
for S in val test
do
CUDA_VISIBLE_DEVICES=$GPUID python save_hidden_states.py \
--dataset_path datasets/BBC/${S}_comprehensive.csv \
--output_dir datasets/BBC/Llama2-7B-Chat_${S} \
--layers_to_process 32 \
--model_name $MODEL \
--max_sample -1 \
--batch_size 16 \
--type last \
--ignore_nan

done
